# Линейная регрессия

Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).


In [38]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
import scipy
from sklearn.linear_model import Ridge

In [2]:
data = pd.read_csv('salary-train.csv')

In [8]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


Проведите предобработку:

Приведите тексты к нижнему регистру (text.lower()).

In [14]:
data['FullDescription'].str.lower()

0        international sales manager london ****k  ****...
1        an ideal opportunity for an individual that ha...
2        online content and brand manager// luxury reta...
3        a great local marketleader is seeking a perman...
4        registered nurse / rgn  nursing home for young...
                               ...                        
59995    as a result of continued growth, first class s...
59996    php / mvc web developer  macclesfieldcirca ***...
59997    staff nurse, nursing home, baldock white recru...
59998    this is one of the best agency side opportunit...
59999    must have cscs card must have asbestos awarene...
Name: FullDescription, Length: 60000, dtype: object

In [13]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager// luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,NaN,20355


Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [15]:
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [16]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [23]:
vectorizer = TfidfVectorizer(min_df = 5)
Matrix = vectorizer.fit_transform(data['FullDescription'])
Matrix

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [25]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [31]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [34]:
X_train = scipy.sparse.hstack([Matrix, X_train_categ])

 Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [39]:
clf = Ridge(alpha=1, random_state=241)

In [40]:
y_train = data['SalaryNormalized']

In [41]:
clf.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.